In [ ]:
# coding:utf-8
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def gragh(result, epoch):    
    x = np.arange(0, epoch, 1)
    left = np.array(x)
    height = np.array(result)
    plt.plot(left, height)

In [ ]:
from deep_qnetwork_with_cnn import DeepQNetworkWithCNN
import random
import time
import copy
import numpy as np

ACTION = [0, 1, 2, 3, 4, 5]
NUM_HID1 = 288
NUM_HID2 = 100
NUM_OUT  = 6
BATCH_SIZE = 50
EPOCH = 1000
GAMMA = 0.9
EPSIL = 0.5
MINEPSIL = 0.1
NUM_LIMIT_RECORD = 1000
NUM_INIT_RECORD = 20000
ProccerUnit = "CPU"
MODEL_NAME = "./DQN_model/breakout1.model"
RESULT_REWARD = []

STATE_LENGTH = 4  # 状態を構成するフレーム数
FRAME_WIDTH = 128  # リサイズ後のフレーム幅
FRAME_HEIGHT = 128  # リサイズ後のフレーム高さ
NO_OP_STEPS = 30

def agent_action(env, action):
    next_state, reward, terminal, info = env.step(action)
    return next_state, reward, terminal, info

def deep_q_learn(env, agent):
    agent.load_weight()
    agent.update_target_network()
    step = 0
    for episode in range(EPOCH):
        sum_reward = 0
        terminal = False  # エピソード終了判定を初期化
        next_obs = env.reset()  # 環境の初期化、初期画面を返す
        for i in range(random.randint(1, NO_OP_STEPS)):  # ランダムなフレーム数分「何もしない」行動で遷移させる
            now_obs = next_obs
            next_obs, _, _, _ = env.step((i%4)+2)  # 「何もしない」行動を取り、次の画面を返す
        now_state = agent.get_initial_state(now_obs, next_obs)  # 初期状態を作る
        last = 5
        while(not terminal):
            # 状態更新
            now_obs = copy.deepcopy(next_obs)
            # 方策に従って行動を選択
            action, _, action_list = agent.policy_egreedy(now_state, agent.Q_network, step)
            # 行動による次状態を観測
            next_obs, reward, term, info = agent_action(env, ACTION[action])
            # ゴールの設定
            terminal, reward = (True, reward) if term else (False, reward)
            if last != info["ale.lives"]:
                reward = -1.0
                terminal = False
            
            preprocess_obs = agent.pre_proccesing(now_obs, next_obs)
            next_state = np.append(now_state[1:, :, :], np.array([preprocess_obs]), axis=0)
            sum_reward += reward
            agent.stock_record(now_state, action, next_state, reward, terminal, action_list)
            terminal = term
            # env.render()
            if terminal:
                break
                
            # 入力状態更新
            now_state = copy.deepcopy(next_state)
            last = info["ale.lives"]
            
            # Experience_Replayによるバッチ学習
            if len(agent.record) > NUM_LIMIT_RECORD:
                agent.init_record()
                if step % 40 == 0:
                    agent.experience_replay()
                    if agent.EPSIL > MINEPSIL: agent.EPSIL -= EPSIL / EPOCH / 5
            if step % 1000 == 0:
                agent.update_target_network()               
            step += 1
            if step % 200 == 0: print("step:" + str(step))

        print(episode, sum_reward, agent.EPSIL, step)
        RESULT_REWARD.append(sum_reward)
        agent.save_weight()
    gragh(RESULT_REWARD, len(RESULT_REWARD))

if __name__=="__main__":
    import gym
    env = gym.make("Breakout-v0")
    agent = DeepQNetworkWithCNN(NUM_HID1, NUM_HID2, NUM_OUT,ProccerUnit, EPSIL, GAMMA, BATCH_SIZE, MODEL_NAME)
    deep_q_learn(env, agent)